In [1]:
import ast
import sqlite3
import json
from langchain.tools import YouTubeSearchTool
from pytube import YouTube

from youtube_transcript_api import YouTubeTranscriptApi
import re
import pandas as pd
from tqdm import tqdm

In [2]:
# Create a connection
conn = sqlite3.connect('../db/sword_hackathon.db')

# Create a cursor
cursor = conn.cursor()

# Execute SQL command
cursor.execute("CREATE TABLE IF NOT EXISTS youtube_content (id INTEGER PRIMARY KEY, title TEXT, thumbnail_url TEXT, transcription TEXT, length INT, views INT, url TEXT)")

# Commit changes and close connection
conn.commit()
conn.close()

In [3]:
# Connect to the SQLite database
conn = sqlite3.connect('../db/sword_hackathon.db')

# Execute the SQL command and convert it to a pandas DataFrame
df_existing = pd.read_sql_query("SELECT * FROM youtube_content", conn)

# Close the connection to the database
conn.close()

# Display the DataFrame
df_existing.iloc[0]

id                                                               1
title            How To Make Butter Chicken At Home | Restauran...
thumbnail_url         https://i.ytimg.com/vi/a03U45jFxOI/hq720.jpg
transcription    [{"text": "[Music]", "start": 5.78, "duration"...
length                                                         328
views                                                     43467092
url              https://www.youtube.com//watch?v=a03U45jFxOI&p...
Name: 0, dtype: object

In [4]:
tool = YouTubeSearchTool()

In [5]:
all_videos = tool.run("machine learning,10")

In [6]:
all_videos = tool.run("xgboost,10")

list_urls = ast.literal_eval(all_videos)
list_urls

['/watch?v=OtD8wVaFm6E&pp=ygUHeGdib29zdA%3D%3D',
 '/watch?v=TyvYZ26alZs&pp=ygUHeGdib29zdA%3D%3D',
 '/watch?v=OQKQHNCVf5k&pp=ygUHeGdib29zdA%3D%3D',
 '/watch?v=hVOuCfJxprI&pp=ygUHeGdib29zdA%3D%3D',
 '/watch?v=ZVFeW798-2I&pp=ygUHeGdib29zdA%3D%3D',
 '/watch?v=GrJP9FLV3FE&pp=ygUHeGdib29zdA%3D%3D',
 '/watch?v=XXHhrlL-FWc&pp=ygUHeGdib29zdA%3D%3D',
 '/watch?v=FakVn1RgDms&pp=ygUHeGdib29zdA%3D%3D',
 '/shorts/_8NJz-00Sgw',
 '/watch?v=8b1JEDvenQU&pp=ygUHeGdib29zdA%3D%3D']

In [9]:
all_subjects = ['kdd conference']

for i in all_subjects:
    all_videos = tool.run(f"{i},10")
    
    list_urls = ast.literal_eval(all_videos)
    
    conn = sqlite3.connect('../db/sword_hackathon.db')
    
    # Create a cursor
    cursor = conn.cursor()
    
    
    for i in tqdm(list_urls):
    
        if f"https://www.youtube.com/{i}" not in df_existing['url'].unique():
            try:
                id = i.split("watch?v=")[1].split("&")[0]
                yt = YouTube(f"https://www.youtube.com/{i}")
            except IndexError:
                continue
            try:
                transcription = YouTubeTranscriptApi.get_transcript(id)
                transcription_json = json.dumps(transcription)
                
                cursor.execute("INSERT INTO youtube_content (title, transcription, thumbnail_url, length, views, URL) VALUES (?, ?, ?, ?, ?, ?)",
                           (yt.title, transcription_json, str(yt.thumbnail_url), yt.length, yt.views, f"https://www.youtube.com/{i}"))
            except:
                continue
        
    
    # Commit changes and close connection
    conn.commit()
    conn.close()


100%|███████████████████████████████████████████| 10/10 [00:09<00:00,  1.11it/s]
